# Aggregate Discharge from Daily to Monthly

In [16]:
# Importing the libraries
import pandas as pd
import seaborn as sns
import numpy as np
import calendar
import datetime
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

from IPython.display import HTML

sns.set()

Import the data

In [17]:
station_name = 'picadacorbo'
DISCHARGE_DAILY = pd.read_csv(f'../data/{station_name}_caudales.csv',parse_dates=['Fecha'],index_col="Fecha",dayfirst=True,na_values="NA")
# Identify the missing data
DISCHARGE_DAILY_date_missing = pd.date_range(start = DISCHARGE_DAILY.index[0].strftime('%Y-%m-%d'), end = DISCHARGE_DAILY.index[-1].strftime('%Y-%m-%d'),freq='D')
# DISCHARGE_DAILY_date_missing = pd.date_range(start = '1980-01-01', end = '2023-03-06',freq='D')
DISCHARGE_DAILY = DISCHARGE_DAILY.reindex(DISCHARGE_DAILY_date_missing,fill_value=None)
DISCHARGE_DAILY.index.name = 'Fecha' # Set index Fecha
DISCHARGE_DAILY.columns = ['Station','Discharge']
DISCHARGE_DAILY = DISCHARGE_DAILY.drop(columns='Station')
HTML(DISCHARGE_DAILY.tail(6).to_html())

,Discharge
Fecha,
2023-05-26,1.515
2023-05-27,1.761
2023-05-28,2.292
2023-05-29,3.021
2023-05-30,2.969
2023-05-31,2.979


### Calculate monthly mean from daily data

First, we define a percentage of missing value. For this exercise, we use a 50% of missing data in the given month

In [18]:
# Percentage of missing data
max_pct_missing = 50

In [19]:
# group the Dataframe in a monthly time scale
GROUPER_DISCHARGE_MONTHLY = DISCHARGE_DAILY.groupby(pd.Grouper(freq='1MS'))

# this function allows to calculate the percentange of missing values and label the dataframe to "missing"
NUMBER_MISSING = GROUPER_DISCHARGE_MONTHLY.apply(lambda x: pd.isnull(x).sum()*100/len(x)).unstack(1)
NUMBER_MISSING = NUMBER_MISSING.to_frame()

# change the column name to "missing"
NUMBER_MISSING.columns = ['number_missing'] 

# this function allows to calculate the percentange of missing values and label the dataframe to "missing"
BOOL_MISSING = GROUPER_DISCHARGE_MONTHLY.apply(lambda x: pd.isnull(x).sum()*100/len(x)).unstack(1) < max_pct_missing
BOOL_MISSING = BOOL_MISSING.to_frame() # Convert to DataFrame
BOOL_MISSING.columns = ['missing'] # change the column name to "missing"

# BOOL_MISSING[~BOOL_MISSING['missing']] # print the dates that does not fulfill the criterion of null data in for each month
# BOOL_MISSING.to_clipboard()

# NUMBER_MISSING.to_clipboard() # Uncomment if you want to visualize all the result in a CSV

Next, we identify the months which contains lower that the max_pct_missing value and the monthly flow is only calculated if 50% o more of recorded value in a given month

In [20]:
# from daily to monthly
DISCHARGE_MONTHLY = DISCHARGE_DAILY.resample('M',closed="right").apply(lambda x: x.mean() if x.isnull().sum()*100/len(x) < max_pct_missing else np.nan)
# from monthly to 3 months
DISCHARGE_THREE_MONTHS = DISCHARGE_MONTHLY.rolling(3).apply(lambda x: x.mean() if x.isnull().sum()*100/len(x) < max_pct_missing else np.nan)
# Create columns
DISCHARGE_MONTHLY['year'] = DISCHARGE_MONTHLY.index.year
DISCHARGE_MONTHLY['month'] = DISCHARGE_MONTHLY.index.month
## create column for day, month, year in the daily discharge ()
DISCHARGE_DAILY['year'] = DISCHARGE_DAILY.index.year
DISCHARGE_DAILY['month'] = DISCHARGE_DAILY.index.month
DISCHARGE_DAILY['day'] = DISCHARGE_DAILY.index.day
DISCHARGE_DAILY['monthday'] = DISCHARGE_DAILY.index.day_of_year
# The water year for this region starts in April (4)
DISCHARGE_DAILY['water_year'] = DISCHARGE_DAILY.index.year.where(DISCHARGE_DAILY.index.month < 4, DISCHARGE_DAILY.index.year + 1)
# Change the day in the datetime column
DISCHARGE_MONTHLY.index = DISCHARGE_MONTHLY.index.map(lambda t: t.replace(day=1))
## print the first results
HTML(DISCHARGE_MONTHLY.tail(12).to_html(index=True))

,Discharge,year,month
Fecha,,,
2022-06-01,7.725967,2022,6
2022-07-01,54.101097,2022,7
2022-08-01,12.203742,2022,8
2022-09-01,6.864467,2022,9
2022-10-01,2.841645,2022,10
2022-11-01,9.050967,2022,11
2022-12-01,1.491739,2022,12
2023-01-01,0.107290,2023,1
2023-02-01,0.001179,2023,2


In [21]:
DISCHARGE_MONTHLY = DISCHARGE_MONTHLY.drop(columns={'year','month'})
DISCHARGE_MONTHLY = DISCHARGE_MONTHLY.rename(columns={'Discharge': 'Caudal'})

In [22]:
DISCHARGE_MONTHLY.to_csv(f'd:/Documentos/R/mensuales_indice_caudales/input/{station_name}_monthly.csv',date_format='%d/%m/%Y')